## Preprocessing data of Ziffra et al., Nature (2021)

In [ ]:
import pandas as pd
import os, glob
from natsort import natsort_keygen

In [ ]:
peaks = pd.read_excel("~/Dropbox/CWAS_paper_WD/Data/raw_annotations/Ziffra2021_TableS2-13.xlsx", sheet_name="ST2 AllPrimaryPeaks")
enhancers = pd.read_excel("~/Dropbox/CWAS_paper_WD/Data/raw_annotations/Ziffra2021_TableS2-13.xlsx", sheet_name="ST5 EnhancerPeaks")

In [ ]:
cell_types = [c.replace("_Enhancerpeaks", "") for c in enhancers.columns.tolist() if "_Enhancerpeaks" in c]
cell_types

In [ ]:
ziffra_dict = {}

for c in cell_types:
    enh_col = c + "_Enhancerpeaks"
    tar_col = c + "_targetgene"
    
    peak_df = enhancers.loc[enhancers[enh_col]==1, ["peak_name", enh_col, tar_col]]
    peak_df = pd.merge(peaks, peak_df, on="peak_name")
    peak_res = peak_df.iloc[:, [0,1,2,5]].sort_values(["seqnames","start"], key=natsort_keygen())
    peak_name = "Ziffra_" + c + "_enhancers.hg38.sorted.bed"
    peak_res.to_csv(os.path.join("~/Dropbox/CWAS_paper_WD/Data/cwas_annotation.v5", peak_name), sep="\t", index=False, header=False)
    
    ziffra_dict[peak_name] = "ZiffraCRE." + c

## Create index file

In [ ]:
all_files = glob.glob("/Users/yujinkim/Dropbox/CWAS_paper_WD/Data/cwas_annotation.v5/*.bed.gz")
len(all_files)

In [ ]:
pre = [z for z in all_files if os.path.basename(z).startswith("Ziffra2021")]
pre

In [ ]:
for f in pre:
    os.system(f"tabix -p bed {f} -f")